<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 1
#### Student Name: Yehezkiel Efraim Darmadi, Yogi Sarumaha
#### Student ID: 34078215, 34087672

Date: 22 August 2024


Environment: Python3

Libraries used:
* re (for regular expression, installed and imported)
* pandas (for data manipulation)
* os (for accessing directories)
* datetime (to deal with date object)
    
</div>

<div class="alert alert-block alert-danger">
    
## Table of Contents

</div>    

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Patent Files](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Defining Regular Expressions](#Reg_Exp) <br>
$\;\;\;\;$[4.2. Reading Files](#Read) <br>
$\;\;\;\;$[4.3. Clean the txt file](#clean_text) <br>
$\;\;\;\;$[4.4. Clean the csv file](#clean_csv) <br>
$\;\;\;\;$[4.5. Combine the txt file and csv file](#combine) <br>
$\;\;\;\;$[4.6. Remove emojis](#remove_emo) <br>
$\;\;\;\;$[4.7. Remove non english translation](#non_eng) <br>
[5. Writing to CSV and JSON File](#write) <br>
$\;\;\;\;$[5.1. Writing the CSV file](#write_csv) <br>
$\;\;\;\;$[5.2. Writing the JSON file](#write_json) <br>
$\;\;\;\;$[5.3. Verification of the Generated JSON File and CSV file](#verification) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

-------------------------------------

<div class="alert alert-block alert-warning">

## 1.  Introduction  <a class="anchor" name="Intro"></a>
    
</div>

This assessment regards extracting data from semi-sctuctured text files. The dataset contained 500 `.txt` files which included various information about user reviews. In particular, this assessment involves extracting data from semi-structured text files. The dataset consists of 15 mis-structured XML formatted .txt files and an Excel file. These files contain various information related to Google Map reviews from businesses in California. Each review is recorded with several attributes such as user_id, name, time, rating, text, pics, resp, and gmap_id. The objective is to parse these files to extract and transform the data into structured formats including a CSV file with specific columns and a JSON file adhering to a given structure.

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

The packages to be used in this assessment are imported in the following. They are used to fulfill the following tasks:

* **re:** to define and use regular expressions
* **pandas:** to wrangle df
* **os:** to list the files in a directory
* **datetime**: to wrangle date type object

In [21]:
pip install openpyxl

In [22]:
import re
import json
import pandas as pd
import os
import datetime

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
file_directory = "/content/drive/Shareddrives/FIT5196_S2_2024/GroupAssessment1/Student Data/student_group111"

-------------------------------------

<div class="alert alert-block alert-warning">

## 3.  Examining Raw Data <a class="anchor" name="examine"></a>

 </div>

First of all, let's try import one txt file and the csv file into the jupyter notebook to check the structure of the data.

In [25]:
!ls "/content/drive/Shareddrives/FIT5196_S2_2024/GroupAssessment1/Student Data/student_group111"

group111_0.txt	 group111_12.txt  group111_1.txt  group111_4.txt  group111_7.txt  group111.xlsx
group111_10.txt  group111_13.txt  group111_2.txt  group111_5.txt  group111_8.txt
group111_11.txt  group111_14.txt  group111_3.txt  group111_6.txt  group111_9.txt


In [26]:
with open(file_directory + "/group111_0.txt", "r", encoding="utf-8") as file:
  content = file.readlines()

content[:10]

['?xml version="1.0" encoding="UTF-8"?>\n',
 '<dataset>\n',
 '<record>\n',
 '<rate>2< /rate>< date>\n',
 '1621090615383< /date>\n',
 '< userid>\n',
 '107155870469692798549<//userid>\n',
 '<Pictures>None<//Pictures>< Name>\n',
 'jamie lee</Name>\n',
 '<review>I came in for a shoulder length cut with face framing layers, and what I got was a very weirdly uneven cut, the hair in front was actually longer than in back, etc. It just did not look good nor resemble the pictures I showed. I think going to a budget salon for anything trendy is not a good idea. Maybe just for simple trims but you really do get what you pay for. The lady who cut my hair was very nice, just think I would’ve gotten a much better cut someplace else.<//review><Response>None</Response><Gmap_id>0x809468ccc55c6627:0xae8dfd844b94d156<//Gmap_id></record>\n']

In [27]:
for fl in os.listdir(file_directory):
  if fl[-3:] != "txt":
    df_xlsx = pd.read_excel(file_directory + "/" + fl)
df_xlsx.head()

,x3,x1,user_id,name,time,rating,text,pics,resp,gmap_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,115518686756006149377,Maryhelen Buentiempo,1.618600e+12,5.0,I love this car wash came in always friendly i...,NaN,NaN,0x80ec6b7e1ab8b429:0xefaf8c17c1e8723c
3,NaN,NaN,106015699523071445097,Debra Coker,1.612217e+12,4.0,"I love this car wash, they are very good and p...",NaN,NaN,0x80ec6b7e1ab8b429:0xefaf8c17c1e8723c
4,NaN,NaN,114959215102122502113,Sharon,1.615322e+12,1.0,I suppose if you have never been to a car wash...,NaN,NaN,0x80ec6b7e1ab8b429:0xefaf8c17c1e8723c


Having examined the file content, the following observations were made:

- The `.txt` file appears to contain semi-structured data in an XML-like format. However, several formatting issues were observed, such as:
  - The presence of extraneous spaces within the tags, such as `< /rate>` and `< /date>`, which should be corrected to `</rate>` and `</date>`.
  - Incorrect closing tags, for example, `<//userid>` and `<//Pictures>` instead of `</userid>` and `</Pictures>`.
  - The tags are inconsistently formatted, which can complicate the parsing and extraction process.
- The `.csv` file includes structured data but contains rows with missing values (represented by `NaN`), particularly in columns like `x3`, `x1`, `user_id`, `name`, `time`, `rating`, `text`, `pics`, `resp`, and `gmap_id`.
- Both the `.txt` and `.csv` files seem to contain user reviews, including information such as `user_id`, `name`, `time`, `rating`, `review text`, `pictures`, and `response`. The `gmap_id` field appears to uniquely identify the business associated with each review.
- The `.txt` file data needs to be properly cleaned and parsed to ensure that all relevant information is accurately extracted and formatted.
- The `.txt` file also seems to have multiple different word format for a feature, for example: "user_id" and "userid".
- There are also translated text that needs to be removed in the `.txt` files.
- There are also emojis that the assignment specifically ask to remove.

-------------------------------------

<div class="alert alert-block alert-warning">

## 4.  Loading and Parsing Files <a class="anchor" name="load"></a>

</div>

In this section, the files are parsed and processed. First of all, appropriate regular expressions are defined to extract desired information when reading the files. The regular expressions are tailored to identify specific tags and their content within the semi-structured text, correcting any formatting inconsistencies explained in the previous section. Once the regular expressions are defined, the text files are iteratively read and parsed, with each record being cleaned and transformed into a structured format. The csv is also cleaned and then it is combined with the txt file structured data.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.1. Defining Regular Expressions <a class="anchor" name="Reg_Exp"></a>

Defining correct regular expressions is crucial in extracting desired information from the text efficiently. ...

In [28]:
def data_extraction(records):
    """
    Extracts key-value pairs from semi-structured text data records.

    The function processes a list of text records, correcting tag formatting issues
    such as extra spaces and incorrect closing tags. It then uses a regular expression
    to find all key-value pairs within tags and stores them in a list of dictionaries
    where keys are tag names in lowercase and values are the corresponding text content.

    Args:
        records (list of str): A list of text records containing tags.

    Returns:
        list of dict: A list of dictionaries where each dictionary contains
                      extracted key-value pairs from the corresponding record.
    """
    raw_records = []
    pattern = r'<(.*?)>(.*?)<\s*/\1>'
    for record in records:
        record = re.sub(r'<//', r'</', record)
        record = re.sub(r'<\s*', r'<', record)
        record = re.sub(r'\s*>', r'>', record)
        raw_record = re.findall(pattern, record, re.DOTALL)
        raw_record = {key.lower(): value.strip() for key, value in raw_record}
        raw_records.append(raw_record)

    return raw_records

These patterns are used in the next step when reading the files. This pattern is used to get the key and the values for every field in the xml file.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.2. Reading Files <a class="anchor" name="Read"></a>

In this step, all files are read and parsed.

In [29]:
raw_records = []
# pattern to get each records
pattern = re.compile(r'<record>(.*?)</record>', re.DOTALL)

# iterating through the file in the directory
for fl in os.listdir(file_directory):
  # selecting only txt file
  if fl[-3:] == "txt":
    with open(file_directory + "/" + fl, "r", encoding="utf-8") as file:
      content = file.read()

    records = pattern.findall(content)

    raw_records += data_extraction(records)

# create df
df_txt = pd.DataFrame(raw_records)
df_txt.head(10)

,rate,date,userid,pictures,name,review,response,gmap_id,gmapid,resp,rating,pics,user_name,text,user,username,time,user_id,userid.
0,2,1621090615383,107155870469692798549,None,jamie lee,I came in for a shoulder length cut with face ...,None,0x809468ccc55c6627:0xae8dfd844b94d156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1503258742655,101193405633692539765,NaN,Kathy Pineda,My son was so sad after we left he starts scho...,NaN,NaN,0x809468ccc55c6627:0xae8dfd844b94d156,"{'time': 1503341994472, 'text': ""Dear Ms. Pine...",1,[{'url': ['https://lh5.googleusercontent.com/p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,1615100544871,101540998139706960282,None,NaN,NaN,NaN,NaN,0x809468ccc55c6627:0xae8dfd844b94d156,None,NaN,NaN,Maricela Gallegos,This salon takes all the precautions necessary...,NaN,NaN,NaN,NaN,NaN
3,5,NaN,NaN,None,NaN,NaN,NaN,NaN,0x809468ccc55c6627:0xae8dfd844b94d156,None,NaN,NaN,NaN,Kerry gave me a fantastic haircut. Thanks. Wil...,107274858014393181439,r v,1622078854258,NaN,NaN
4,1,NaN,NaN,NaN,Cam The ham,NaN,NaN,NaN,0x809468ccc55c6627:0xae8dfd844b94d156,None,NaN,None,NaN,I went in for a haircut. Medical conditions pr...,NaN,NaN,1620708097410,109832829112851321601,NaN
5,NaN,NaN,106666375227252832172,NaN,Debbie Morales,Went in for haircut fast service. They were ve...,NaN,0x809468ccc55c6627:0xae8dfd844b94d156,NaN,None,5,None,NaN,NaN,NaN,NaN,1620860792009,NaN,NaN
6,1,NaN,NaN,NaN,PreetoBehindThe Phone,NaN,NaN,0x809468ccc55c6627:0xae8dfd844b94d156,NaN,None,NaN,None,NaN,Went to do my haircut I told her I needed laye...,110415177023919394002,NaN,1615354538017,NaN,NaN
7,NaN,1591668413727,NaN,None,Donna Shepard,My son and I were able to get in quickly. They...,"{'time': 1600118239289, 'text': 'We are trying...",0x809468ccc55c6627:0xae8dfd844b94d156,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103343421447344432052
8,5,1573333822470,103851429787157060044,NaN,Jorge Gato,Great location great service. Nora was profe...,NaN,NaN,0x809468ccc55c6627:0xae8dfd844b94d156,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5,NaN,NaN,NaN,NaN,NaN,None,0x809468ccc55c6627:0xae8dfd844b94d156,NaN,NaN,NaN,None,NaN,Friendly professional staff great location. Ki...,NaN,Peggy Harshman,1563466022318,110161133072090009549,NaN


Let's take a look at the first ten elements of the lists generated. We can see that ids, reviews,etc. are parsed and stored correctly.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.3. Clean the txt file <a class="anchor" name="clean_text"></a>

This section will clean the df that we got from the txt files, it is more focused on combining and fixing the column names and standardise it.

In [30]:
# fixing user_id
df_txt["user_id"] = df_txt["user_id"].fillna(df_txt["userid"])
df_txt["user_id"] = df_txt["user_id"].fillna(df_txt["userid."])
df_txt["user_id"] = df_txt["user_id"].fillna(df_txt["user"])
df_txt.drop(columns=["userid", "userid.", "user"], inplace = True)

# fixing name
df_txt["name"] = df_txt["name"].fillna(df_txt["user_name"])
df_txt["name"] = df_txt["name"].fillna(df_txt["username"])
df_txt.drop(columns=["user_name", "username"], inplace = True)

# fixing time
df_txt["time"] = df_txt["time"].fillna(df_txt["date"])
df_txt.drop(columns=["date"], inplace = True)
df_txt['time'] = df_txt['time'].astype('float64')

# fixing rating
df_txt["rating"] = df_txt["rating"].fillna(df_txt["rate"])
df_txt.drop(columns=["rate"], inplace = True)
df_txt['rating'] = df_txt['rating'].astype('float64')
# df_txt['rating'] = df_txt['rating'].fillna("None")

# fixing text
df_txt["text"] = df_txt["text"].fillna(df_txt["review"])
df_txt.drop(columns=["review"], inplace = True)

# fixing pics
df_txt["pics"] = df_txt["pics"].fillna(df_txt["pictures"])
df_txt.drop(columns=["pictures"], inplace = True)

# fixing resp
df_txt["resp"] = df_txt["resp"].fillna(df_txt["response"])
df_txt.drop(columns=["response"], inplace = True)

# fixing pics
df_txt["gmap_id"] = df_txt["gmap_id"].fillna(df_txt["gmapid"])
df_txt.drop(columns=["gmapid"], inplace = True)

# fixing the df
df_txt.replace({"None": pd.NA}, inplace = True)

df_txt.head(10)

,name,gmap_id,resp,rating,pics,text,time,user_id
0,jamie lee,0x809468ccc55c6627:0xae8dfd844b94d156,<NA>,2.0,<NA>,I came in for a shoulder length cut with face ...,1.621091e+12,107155870469692798549
1,Kathy Pineda,0x809468ccc55c6627:0xae8dfd844b94d156,"{'time': 1503341994472, 'text': ""Dear Ms. Pine...",1.0,[{'url': ['https://lh5.googleusercontent.com/p...,My son was so sad after we left he starts scho...,1.503259e+12,101193405633692539765
2,Maricela Gallegos,0x809468ccc55c6627:0xae8dfd844b94d156,<NA>,5.0,<NA>,This salon takes all the precautions necessary...,1.615101e+12,101540998139706960282
3,r v,0x809468ccc55c6627:0xae8dfd844b94d156,<NA>,5.0,<NA>,Kerry gave me a fantastic haircut. Thanks. Wil...,1.622079e+12,107274858014393181439
4,Cam The ham,0x809468ccc55c6627:0xae8dfd844b94d156,<NA>,1.0,<NA>,I went in for a haircut. Medical conditions pr...,1.620708e+12,109832829112851321601
5,Debbie Morales,0x809468ccc55c6627:0xae8dfd844b94d156,<NA>,5.0,<NA>,Went in for haircut fast service. They were ve...,1.620861e+12,106666375227252832172
6,PreetoBehindThe Phone,0x809468ccc55c6627:0xae8dfd844b94d156,<NA>,1.0,<NA>,Went to do my haircut I told her I needed laye...,1.615355e+12,110415177023919394002
7,Donna Shepard,0x809468ccc55c6627:0xae8dfd844b94d156,"{'time': 1600118239289, 'text': 'We are trying...",5.0,<NA>,My son and I were able to get in quickly. They...,1.591668e+12,103343421447344432052
8,Jorge Gato,0x809468ccc55c6627:0xae8dfd844b94d156,<NA>,5.0,<NA>,Great location great service. Nora was profe...,1.573334e+12,103851429787157060044
9,Peggy Harshman,0x809468ccc55c6627:0xae8dfd844b94d156,<NA>,5.0,<NA>,Friendly professional staff great location. Ki...,1.563466e+12,110161133072090009549


-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.4. Clean the csv file <a class="anchor" name="clean_csv"></a>

In this step, we are going to clean the csv file since we have imported it in the previous section.

In [31]:
# preprocessed the csv file
df_xlsx.drop(columns=["x3", "x1"], inplace = True)
df_xlsx = df_xlsx.dropna(how = "all")

df_xlsx.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
2,115518686756006149377,Maryhelen Buentiempo,1.618600e+12,5.0,I love this car wash came in always friendly i...,NaN,NaN,0x80ec6b7e1ab8b429:0xefaf8c17c1e8723c
3,106015699523071445097,Debra Coker,1.612217e+12,4.0,"I love this car wash, they are very good and p...",NaN,NaN,0x80ec6b7e1ab8b429:0xefaf8c17c1e8723c
4,114959215102122502113,Sharon,1.615322e+12,1.0,I suppose if you have never been to a car wash...,NaN,NaN,0x80ec6b7e1ab8b429:0xefaf8c17c1e8723c
5,116863697481431417830,jeff Steinfurth,1.503789e+12,1.0,Car looks fine but the place is falling apart ...,[{'url': ['https://lh5.googleusercontent.com/p...,NaN,0x80ec6b7e1ab8b429:0xefaf8c17c1e8723c
6,110690235519593823796,El Niño,1.616484e+12,4.0,Good-above average car wash and detail.\nThe G...,NaN,NaN,0x80ec6b7e1ab8b429:0xefaf8c17c1e8723c


-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.5. Combine the txt file and the csv file <a class="anchor" name="combine"></a>

In this section, we are going to combine the txt and csv file into a single df.

In [32]:
# combine both the df_txt and df_xlsx while fixing some issues
df = pd.concat([df_txt, df_xlsx])
# remove duplicate
df.drop_duplicates(inplace = True)
# reseting the index
df.reset_index(inplace = True)
df['time'] = pd.to_datetime(df['time'], unit='ms', utc=True).dt.strftime('%Y-%m-%d %H:%M:%S')
df['text'] = df['text'].fillna("None")
df['text'] = df['text'].str.strip('[]')
df['resp'] = df['resp'].fillna("None")
df['pics'] = df['pics'].fillna("None")

df.head()

,index,name,gmap_id,resp,rating,pics,text,time,user_id
0,0,jamie lee,0x809468ccc55c6627:0xae8dfd844b94d156,None,2.0,None,I came in for a shoulder length cut with face ...,2021-05-15 14:56:55,107155870469692798549
1,1,Kathy Pineda,0x809468ccc55c6627:0xae8dfd844b94d156,"{'time': 1503341994472, 'text': ""Dear Ms. Pine...",1.0,[{'url': ['https://lh5.googleusercontent.com/p...,My son was so sad after we left he starts scho...,2017-08-20 19:52:22,101193405633692539765
2,2,Maricela Gallegos,0x809468ccc55c6627:0xae8dfd844b94d156,None,5.0,None,This salon takes all the precautions necessary...,2021-03-07 07:02:24,101540998139706960282
3,3,r v,0x809468ccc55c6627:0xae8dfd844b94d156,None,5.0,None,Kerry gave me a fantastic haircut. Thanks. Wil...,2021-05-27 01:27:34,107274858014393181439
4,4,Cam The ham,0x809468ccc55c6627:0xae8dfd844b94d156,None,1.0,None,I went in for a haircut. Medical conditions pr...,2021-05-11 04:41:37,109832829112851321601


Data cleansing is a better to do it after we merge the two dataset together, instead of doing it one by one.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.6. Remove emojis <a class="anchor" name="remove_emo"></a>

This section will remove the required emojis that we need to remove from the data.

In [33]:
def clean_text(text):
    """
    Removes emojis from the input text and converts it to lowercase UTF-8 encoded string.

    The function uses a regular expression to identify and remove various emojis,
    including emoticons, symbols, transport/map symbols, and flags. The cleaned text
    is then converted to lowercase and re-encoded as UTF-8.

    Args:
        text (str): The input text from which emojis should be removed.

    Returns:
        str: The cleaned text in lowercase and encoded in UTF-8.
    """
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"
                                u"\U0001F300-\U0001F5FF"
                                u"\U0001F680-\U0001F6FF"
                                u"\U0001F1E0-\U0001F1FF"
                                "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    return text.lower().encode('utf-8').decode('utf-8')

df['text'] = df['text'].apply(lambda x: clean_text(x))

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.7. Remove non english translation text <a class="anchor" name="non_eng"></a>

This section will only keep the english translated text. The text is indicated by "(Translation by google)" in the beginning of the text.

In [34]:
def extract_english_translation(text):
    """
    Extracts the English translation from a text marked by specific tags.

    The function identifies and extracts text segments marked as translations
    using a regular expression that looks for text between the markers
    '(translated by google)' and '(original)'. It then cleans up the extracted
    text by stripping leading/trailing spaces, removing newlines, and removing
    emojis using the `clean_text` function.

    Args:
        text (str): The input text containing English translations marked by specific tags.

    Returns:
        list of str: A list of cleaned English translations. If no matches are found,
                     returns a list containing the original text.
    """
    pattern = r'\(translated by google\)(.*?)\(original\)'
    matches = re.findall(pattern, text, re.DOTALL)
    translations = [clean_text(match.strip()) for match in matches]

    if not translations:
        return [text]

    return translations

df['text'] = df['text'].apply(lambda x: extract_english_translation(x))

-------------------------------------

<div class="alert alert-block alert-warning">

## 5.  Writing to CSV and JSON Files
</div>

This section will detail the steps involved in preparing the required output files for the assignment, specifically focusing on generating the CSV and JSON files. The process includes extracting and transforming the raw data from the provided semi-structured text files and Excel file, cleaning and organizing the data into a structured format, and finally, saving the processed data into the specified CSV and JSON formats as outlined in the assignment requirements.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.1. Writing the csv file <a class="anchor" name="write_csv"></a>

This section focuses on the process of generating the required CSV output file from the extracted data. The CSV file must include specific columns such as gmap_id, review_count, review_text_count, and response_count, which summarize the key aspects of the reviews for each business. The steps include aggregating and counting relevant data points from the parsed text and Excel files, formatting them according to the assignment’s specifications, and then saving the structured data into a CSV file. Special care is taken to ensure the data is accurately captured and the output file adheres to the required format.

In [35]:
df_csv = df.groupby("gmap_id").agg(
    review_count = ("gmap_id", "size"),
    review_text_count=('text', lambda x: sum([1 for i in x if i != ["none"]])),
    response_count=('resp', lambda x: (x != "None").sum())
).reset_index()
df_csv.head(10)

,gmap_id,review_count,review_text_count,response_count
0,0x14e4bcd95f3c0451:0x7ccf04478a4d59af,216,151,7
1,0x54ce1e8992fba04b:0x74eb10642cf7f893,87,52,4
2,0x54d293327ff446fd:0x9e0b0346f57c02fb,67,45,4
3,0x54d2ecbe38c5fc21:0x236ca48a33a9e643,55,47,0
4,0x54d2ecbe5ab770fb:0x7b53821f6d820cb0,64,33,0
5,0x808283476effb1a9:0xdca2d9328e14bc04,138,95,0
6,0x8083289e2f7b9da3:0x40d88c45cc3b6489,55,20,0
7,0x808417340caeb2f5:0xe5127979a5ee695f,75,49,0
8,0x80843de326781dc7:0xf5f15ad461293420,55,29,0
9,0x80843ef5be3f0aa1:0x4104e892e0f462b1,111,71,8


-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.2. Writing the json file <a class="anchor" name="write json"></a>

This section describes the procedure for generating the JSON output file as required by the assignment. The JSON file needs to encapsulate detailed information for each business, including all reviews, their respective attributes (e.g., `user_id`, `time`, `review_rating`, `review_text`), and metadata such as `earliest_review_date` and `latest_review_date`. The process involves carefully structuring the extracted data into a hierarchical format that conforms to the JSON schema provided in the assignment guidelines. This ensures that each business's review data is accurately represented and that the output file meets the specified formatting requirements, including the proper naming conventions and the use of UTC format for date and time fields.

In [36]:
def extract_dimensions(pics):
    """
    Extracts image dimensions from a list of picture URLs.

    The function processes a JSON-formatted string containing a list of picture URLs,
    searching for dimensions formatted as 'w<width>-h<height>' within each URL.
    It extracts these width and height values and stores them as pairs in a list.

    Args:
        pics (str): A JSON-formatted string containing a list of dictionaries with picture URLs.

    Returns:
        list of list: A list of [width, height] pairs extracted from the picture URLs.
    """
    dimensions = []
    pattern = re.compile(r'w(\d+)-h(\d+)')
    pic_list = json.loads(pics.replace("'", '"'))

    for pic in pic_list:
        for url in pic['url']:
            match_pattern = pattern.search(url)
            if match_pattern:
                dimensions.append([match_pattern.group(1), match_pattern.group(2)])

    return dimensions

In [37]:
json_output = {}
for gmap_id, group in df.groupby('gmap_id'):
    reviews_list = group.apply(lambda row: {
        'user_id': row['user_id'],
        'time': row["time"],
        'review_rating': row['rating'],
        'review_text': row["text"][0] if row["text"] else "None",
        'if_pic': 'Y' if row['pics'] != "None" and row['pics'] else 'N',
        'pic_dim': [] if row['pics'] == "None" else extract_dimensions(row['pics']),
        'if_response': 'Y' if row['resp'] != "None" and row['resp'] else 'N'
    }, axis=1).tolist()

    earliest_review_date = min(group['time'])
    latest_review_date = max(group['time'])

    business_data = {
        'reviews': reviews_list,
        'earliest_review_date': earliest_review_date,
        'latest_review_date': latest_review_date
    }

    json_output[gmap_id] = business_data

-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.3. Verification of the Generated JSON File and CSV file <a class="anchor" name="verification"></a>

This section will validate the structure of the generated JSON file and the CSV file against the required standard and then proceed to export the file.

In [38]:
output_dir = "/content/drive/MyDrive/Data Wrangling/assignment 1/Task 1"

with open(output_dir + '/task1_111.json', 'w') as json_file:
    json.dump(json_output, json_file, indent=4)

In [39]:
df_csv.to_csv(output_dir + '/task1_111.csv', index=False)

Please input your group number:111

Task 1 json file passed!

Task 1 csv file passed!

-------------------------------------

<div class="alert alert-block alert-warning">

## 6. Summary <a class="anchor" name="summary"></a>

</div>

In this task, we processed semi-structured text files containing user reviews from businesses in California. The data required extensive cleaning and parsing due to various formatting inconsistencies, such as incorrect XML tags, presence of non-English translations, and emojis.

We began by importing the necessary libraries and examining the raw data. Regular expressions were defined to correct and extract relevant information from the text files. The text data was then parsed and cleaned, including fixing inconsistent tags, removing non-English text, and eliminating emojis. Similarly, the accompanying CSV file was cleaned to ensure consistency with the parsed text data.

After preprocessing, we combined the cleaned text data and CSV data into a single DataFrame. Further data cleansing was performed on the combined dataset to standardize various fields, such as user IDs, names, timestamps, and ratings.

Finally, we generated the required output files: a CSV file summarizing key review metrics for each business and a JSON file containing detailed information on each review. Both output files were validated against the assignment requirements, ensuring accuracy and adherence to the specified formats.

This process involved intricate data wrangling techniques to transform the raw semi-structured data into well-organized, structured formats, fulfilling the objectives of the task.

-------------------------------------

<div class="alert alert-block alert-warning">

## 7. References <a class="anchor" name="Ref"></a>

</div>



[1]<a class="anchor" name="ref-2"></a> Why do I need to add DOTALL to python regular expression to match new line in raw string, https://stackoverflow.com/questions/22610247, Accessed 30/08/2022.

....


## --------------------------------------------------------------------------------------------------------------------------